# 라이브러리 설치

In [1]:
#!pip install --upgrade selenium

In [2]:
#!pip install -U jupyter

In [3]:
import selenium
selenium.__version__

'4.15.2'

In [4]:
#!pip install -U selenium

# 라이브러리 로드

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import time
import openpyxl
import requests

In [6]:
import pickle
from tqdm import tqdm

In [7]:
######################제공하는 코드 건드리지 마세요 #############################

#크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager

# 브라우저 꺼짐 방지
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('detach', True)
User_Agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36"
chrome_options.add_argument(f"user-agent={User_Agent}")

# 불필요한 에러 메세지 없애기
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

# 크롬 드라이버 경로 지정
#driver_path = "C:/Users/yurry/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe"

# 크롬 드라이버 생성
driver = webdriver.Chrome(options=chrome_options)

In [8]:
#엑셀파일 생성
wb = openpyxl.Workbook("여기어때_가평_1234펜션.xlsx")
ws = wb.create_sheet("시트명")
ws.append(['별점','리뷰'])  #컬럼명 제공

## 펜션 별로 사이트 접속

In [9]:
# 대상이 되는 펜션의 리뷰 링크를 main_url로 설정
main_url = 'https://www.goodchoice.kr/product/detail?ano=63357&adcno=3&sel_date=2023-11-17&sel_date2=2023-11-18'
response = requests.get(main_url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

In [10]:
# 리뷰 한페이지당 고객 10명의 리뷰가 담겨 있음
# 고객 10명에 대한 리뷰 정보를 받아오는 함수
def customer_info():
    for j in range(0,10):

        #별점
        try:
            _star = driver.find_element(By.CSS_SELECTOR,f'#review > ul > li:nth-child({j+1}) > div > div.score_wrap_sm > div.num').text
        except:
            _star="없음"

        #리뷰내용
        try:
            _text = driver.find_element(By.CSS_SELECTOR,f'#review > ul > li:nth-child({j+1}) > div > div.txt').text
        except:
            _text="없음"

        #엑셀 데이터 쌓기
        ws.append([_star,_text])
        time.sleep(1)

In [11]:
driver.implicitly_wait(5) # 웹페이지 로딩 될때까지 5초는 기다림
driver.get(main_url)
time.sleep(3) # 3초

rv = driver.find_element(By.CSS_SELECTOR, '#content > div.tab > button.tab_review')
rv.click() # 리뷰 란으로 이동
time.sleep(3)

# 리뷰 하단 끝까지 스크롤(함수이용)
before_h = driver.execute_script("return window.scrollY")
# 끝까지 스크롤( 끝까지 도달하면 스크롤 중단 )
while True:
    driver.find_element(By.CSS_SELECTOR,"body").send_keys(Keys.END)

    time.sleep(1)

    #스크롤 후 높이
    after_h = driver.execute_script("return window.scrollY")

    #스크롤 값이 같으면 스크롤 멈춤
    if after_h == before_h:
        break
    before_h = after_h


last_page=False

for k in range(1, 101):
    # 마지막 페이지면 종료
    if last_page == True:
        break # 현재 last_page = False임
        # 크롤링
    if k < 6:
        try:
            customer_info() # 우선적으로 크롤링 실시
            # 다음 페이지를 클릭하기 수행(if문을 사용)
            if k != 5:
                next_page = driver.find_element(By.CSS_SELECTOR, f'#pagination > div > button:nth-child({k+1})') # 다음페이지
                next_page.click() # k가 1~4면 반복
            elif k == 5:
                next_button = driver.find_element(By.CSS_SELECTOR, '#pagination > div > button.next')
                next_button.click()  # k가 5면 단순히 다음 클릭이 아닌 화살표 클릭까지 마무리
        except:
            last_page = True # 마지막 페이지라고 break 시키기
                
    else:  # 다음페이지부터는 규칙을 이용하기
        k_mod = (k%5)
        try:
            customer_info() # 일단 크롤링
            if k_mod != 0:
                next_page = driver.find_element(By.CSS_SELECTOR, f'#pagination > div > button:nth-child({k_mod+2})')
                next_page.click()
            elif k_mod == 0:
                next_button = driver.find_element(By.CSS_SELECTOR, '#pagination > div > button.next')
                next_button.click()  # k가 5면 단순히 다음 클릭이 아닌 화살표 클릭까지 마무리
        except:
            last_page = True


driver.quit()

In [12]:
wb.save("여기어때_가평_1234펜션.xlsx")